# Predicción de Expansión de Cobertura 5G en Colombia (2025-2028)

Modelo basado en patrones históricos de adopción de tecnologías móviles (2G→3G→4G→5G).

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings('ignore')

## Carga y Análisis de Patrones Históricos

In [2]:
df = pd.read_csv('data/cobertura_movil_con_datos_geograficos.csv', low_memory=False)

df_agg = df.groupby(['AÑO', 'TRIMESTRE', 'DEPARTAMENTO', 'PROVEEDOR']).agg({
    'COBERTURA 2G': lambda x: (x == 'S').sum(),
    'COBERTURA 3G': lambda x: (x == 'S').sum(),
    'COBERTURA HSPA+, HSPA+DC': lambda x: (x == 'S').sum(),
    'COBERTUTA 4G': lambda x: (x == 'S').sum(),
    'COBERTURA LTE': lambda x: (x == 'S').sum(),
    'CENTRO POBLADO': 'count'
}).reset_index()

df_agg.columns = ['AÑO', 'TRIMESTRE', 'DEPARTAMENTO', 'PROVEEDOR', 'Centros_2G', 'Centros_3G', 'Centros_HSPA', 'Centros_4G', 'Centros_LTE', 'Total_Centros']

print(f"Datos cargados: {df_agg.shape}")
print(f"Años disponibles: {sorted(df_agg['AÑO'].unique())}")

Datos cargados: (5708, 10)
Años disponibles: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]


In [3]:
df_tech = df_agg.groupby('AÑO')[['Centros_2G', 'Centros_3G', 'Centros_HSPA', 'Centros_4G', 'Centros_LTE']].sum().reset_index()

fig = go.Figure()

tecnologias = {
    'Centros_2G': '2G',
    'Centros_3G': '3G', 
    'Centros_HSPA': 'HSPA+',
    'Centros_4G': '4G',
    'Centros_LTE': 'LTE'
}

for col, name in tecnologias.items():
    fig.add_trace(go.Scatter(
        x=df_tech['AÑO'],
        y=df_tech[col],
        mode='lines+markers',
        name=name,
        line=dict(width=3),
        marker=dict(size=8)
    ))

fig.update_layout(
    title='Evolución Histórica de Tecnologías Móviles en Colombia',
    xaxis_title='Año',
    yaxis_title='Centros Poblados con Cobertura',
    height=600,
    hovermode='x unified'
)

fig.write_html('graficos/evolucion_tecnologias_historica.html')
fig.show()

print("\nEstadísticas por tecnología:")
print(df_tech)


Estadísticas por tecnología:
    AÑO  Centros_2G  Centros_3G  Centros_HSPA  Centros_4G  Centros_LTE
0  2015        6935        5190          2810           0          745
1  2016       27760       21236         13158           0         3523
2  2017       24443       18504         12311           0         4337
3  2018       21420       16246         10350           0         6850
4  2019       21812       17512         11975           0        10527
5  2020       25351       21648         15193           0        14657
6  2021       35221       36114         31104       19367         4239
7  2022       33912       37522         34941       31266            0
8  2023       20377       25547         23745       24481            0


## Análisis de Curvas de Adopción

Cada tecnología sigue un patrón de curva S (sigmoid):
1. **Introducción**: Crecimiento lento inicial
2. **Crecimiento**: Aceleración exponencial
3. **Madurez**: Saturación y estabilización
4. **Declive**: Sustitución por nueva tecnología

In [4]:
def calcular_años_desde_inicio(df_tech, tecnologia):
    df_tech_copy = df_tech.copy()
    valores = df_tech_copy[tecnologia].values
    
    primer_año_activo = None
    for i, (año, val) in enumerate(zip(df_tech_copy['AÑO'], valores)):
        if val > 0:
            primer_año_activo = año
            break
    
    if primer_año_activo is None:
        return pd.Series([None] * len(df_tech_copy))
    
    años_desde_inicio = df_tech_copy['AÑO'] - primer_año_activo
    return años_desde_inicio

df_tech['Años_2G'] = calcular_años_desde_inicio(df_tech, 'Centros_2G')
df_tech['Años_3G'] = calcular_años_desde_inicio(df_tech, 'Centros_3G')
df_tech['Años_4G'] = calcular_años_desde_inicio(df_tech, 'Centros_4G')
df_tech['Años_LTE'] = calcular_años_desde_inicio(df_tech, 'Centros_LTE')

print("Años desde inicio de cada tecnología:")
print(df_tech[['AÑO', 'Años_2G', 'Años_3G', 'Años_4G', 'Años_LTE']])

Años desde inicio de cada tecnología:
    AÑO  Años_2G  Años_3G  Años_4G  Años_LTE
0  2015        0        0       -6         0
1  2016        1        1       -5         1
2  2017        2        2       -4         2
3  2018        3        3       -3         3
4  2019        4        4       -2         4
5  2020        5        5       -1         5
6  2021        6        6        0         6
7  2022        7        7        1         7
8  2023        8        8        2         8


## Modelo de Predicción 5G

Estrategia: Usar el patrón de crecimiento de 4G/LTE (tecnologías más recientes) como base para predecir 5G.

In [5]:
df_4g_growth = df_tech[df_tech['Centros_4G'] > 0].copy()
df_4g_growth['Años_desde_inicio'] = range(len(df_4g_growth))

años_historicos = df_4g_growth['Años_desde_inicio'].values
valores_4g = df_4g_growth['Centros_4G'].values

X_4g = años_historicos.reshape(-1, 1)
y_4g = valores_4g

modelo_4g = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
modelo_4g.fit(X_4g, y_4g)

print("Patrón de crecimiento 4G capturado")
print(f"R² del modelo base: {modelo_4g.score(X_4g, y_4g):.4f}")

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=años_historicos,
    y=valores_4g,
    mode='markers',
    name='4G Real',
    marker=dict(size=10, color='blue')
))

años_fit = np.linspace(0, max(años_historicos), 100)
valores_fit = modelo_4g.predict(años_fit.reshape(-1, 1))

fig.add_trace(go.Scatter(
    x=años_fit,
    y=valores_fit,
    mode='lines',
    name='Curva Ajustada',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='Curva de Adopción 4G - Modelo Base para 5G',
    xaxis_title='Años desde inicio',
    yaxis_title='Centros con Cobertura',
    height=500
)

fig.write_html('graficos/curva_adopcion_4g.html')
fig.show()

Patrón de crecimiento 4G capturado
R² del modelo base: 1.0000


In [6]:
año_inicio_5g = 2025
años_prediccion = [2025, 2026, 2027, 2028]
años_desde_inicio_5g = [0, 1, 2, 3]

factor_aceleracion = 1.3

predicciones_5g_base = modelo_4g.predict(np.array(años_desde_inicio_5g).reshape(-1, 1))
predicciones_5g_ajustadas = predicciones_5g_base * factor_aceleracion

df_pred_5g = pd.DataFrame({
    'AÑO': años_prediccion,
    'Años_desde_inicio': años_desde_inicio_5g,
    'Centros_5G_predicho': predicciones_5g_ajustadas
})

df_pred_5g['Centros_5G_predicho'] = df_pred_5g['Centros_5G_predicho'].clip(lower=0).astype(int)

print("\nPredicciones de Expansión 5G:")
print(df_pred_5g)

print("\nResumen de proyecciones:")
for _, row in df_pred_5g.iterrows():
    print(f"  {int(row['AÑO'])}: {int(row['Centros_5G_predicho']):,} centros con 5G")


Predicciones de Expansión 5G:
    AÑO  Años_desde_inicio  Centros_5G_predicho
0  2025                  0                25177
1  2026                  1                40645
2  2027                  2                31825
3  2028                  3                31825

Resumen de proyecciones:
  2025: 25,177 centros con 5G
  2026: 40,645 centros con 5G
  2027: 31,825 centros con 5G
  2028: 31,825 centros con 5G


In [7]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_tech['AÑO'],
    y=df_tech['Centros_4G'],
    mode='lines+markers',
    name='4G (Real)',
    line=dict(color='blue', width=3),
    marker=dict(size=10)
))

fig.add_trace(go.Scatter(
    x=df_pred_5g['AÑO'],
    y=df_pred_5g['Centros_5G_predicho'],
    mode='lines+markers',
    name='5G (Predicción)',
    line=dict(color='red', width=3, dash='dash'),
    marker=dict(size=10, symbol='star')
))

fig.update_layout(
    title='Proyección de Expansión 5G en Colombia (2025-2028)',
    xaxis_title='Año',
    yaxis_title='Centros Poblados con Cobertura',
    height=600,
    hovermode='x unified'
)

fig.write_html('graficos/prediccion_5g_nacional.html')
fig.show()

## Predicciones por Departamento

In [8]:
top_depts = df_agg.groupby('DEPARTAMENTO')['Total_Centros'].sum().nlargest(8).index

df_dept_4g = df_agg[df_agg['AÑO'] >= 2021].groupby(['DEPARTAMENTO', 'AÑO'])['Centros_4G'].sum().reset_index()

predicciones_dept = []

for dept in top_depts:
    df_dept = df_dept_4g[df_dept_4g['DEPARTAMENTO'] == dept].copy()
    
    if len(df_dept) < 2:
        continue
    
    total_4g_dept = df_dept['Centros_4G'].sum()
    proporcion_dept = total_4g_dept / df_tech[df_tech['AÑO'] >= 2021]['Centros_4G'].sum()
    
    for _, row_pred in df_pred_5g.iterrows():
        predicciones_dept.append({
            'DEPARTAMENTO': dept,
            'AÑO': int(row_pred['AÑO']),
            'Centros_5G_predicho': int(row_pred['Centros_5G_predicho'] * proporcion_dept)
        })

df_pred_5g_dept = pd.DataFrame(predicciones_dept)

print("\nPredicciones 5G por Departamento (Top 8):")
print(df_pred_5g_dept.pivot(index='DEPARTAMENTO', columns='AÑO', values='Centros_5G_predicho'))


Predicciones 5G por Departamento (Top 8):
AÑO              2025  2026  2027  2028
DEPARTAMENTO                           
ANTIOQUIA        2866  4627  3623  3623
BOLÍVAR          1063  1717  1344  1344
BOYACÁ           1770  2858  2238  2238
CAUCA             718  1159   907   907
CUNDINAMARCA     3236  5224  4090  4090
CÓRDOBA          1568  2532  1982  1982
NARIÑO            876  1414  1107  1107
VALLE DEL CAUCA  1162  1877  1469  1469


In [9]:
fig = go.Figure()

for dept in top_depts:
    df_hist_dept = df_dept_4g[df_dept_4g['DEPARTAMENTO'] == dept]
    df_pred_dept = df_pred_5g_dept[df_pred_5g_dept['DEPARTAMENTO'] == dept]
    
    fig.add_trace(go.Scatter(
        x=df_hist_dept['AÑO'],
        y=df_hist_dept['Centros_4G'],
        mode='lines+markers',
        name=f'{dept} (4G)',
        line=dict(width=2)
    ))
    
    fig.add_trace(go.Scatter(
        x=df_pred_dept['AÑO'],
        y=df_pred_dept['Centros_5G_predicho'],
        mode='lines+markers',
        name=f'{dept} (5G pred)',
        line=dict(width=2, dash='dash')
    ))

fig.update_layout(
    title='Proyección 5G por Departamento - Top 8',
    xaxis_title='Año',
    yaxis_title='Centros con Cobertura',
    height=700,
    hovermode='x unified'
)

fig.write_html('graficos/prediccion_5g_departamentos.html')
fig.show()

## Comparación de Velocidades de Adopción

In [10]:
techs_comp = pd.DataFrame({
    'Tecnología': ['3G', '4G', '5G (Proyección)'],
    'Año_1': [
        df_tech[df_tech['Centros_3G'] > 0].iloc[0]['Centros_3G'],
        df_tech[df_tech['Centros_4G'] > 0].iloc[0]['Centros_4G'],
        df_pred_5g.iloc[0]['Centros_5G_predicho']
    ],
    'Año_2': [
        df_tech[df_tech['Centros_3G'] > 0].iloc[1]['Centros_3G'] if len(df_tech[df_tech['Centros_3G'] > 0]) > 1 else None,
        df_tech[df_tech['Centros_4G'] > 0].iloc[1]['Centros_4G'] if len(df_tech[df_tech['Centros_4G'] > 0]) > 1 else None,
        df_pred_5g.iloc[1]['Centros_5G_predicho']
    ],
    'Año_3': [
        df_tech[df_tech['Centros_3G'] > 0].iloc[2]['Centros_3G'] if len(df_tech[df_tech['Centros_3G'] > 0]) > 2 else None,
        df_tech[df_tech['Centros_4G'] > 0].iloc[2]['Centros_4G'] if len(df_tech[df_tech['Centros_4G'] > 0]) > 2 else None,
        df_pred_5g.iloc[2]['Centros_5G_predicho']
    ]
})

fig = go.Figure()

for tech in techs_comp['Tecnología']:
    row = techs_comp[techs_comp['Tecnología'] == tech].iloc[0]
    valores = [row['Año_1'], row['Año_2'], row['Año_3']]
    
    fig.add_trace(go.Scatter(
        x=[1, 2, 3],
        y=valores,
        mode='lines+markers',
        name=tech,
        line=dict(width=3),
        marker=dict(size=10)
    ))

fig.update_layout(
    title='Comparación de Velocidad de Adopción por Tecnología',
    xaxis_title='Años desde introducción',
    yaxis_title='Centros con Cobertura',
    height=600,
    xaxis=dict(tickmode='linear', tick0=1, dtick=1)
)

fig.write_html('graficos/comparacion_velocidad_adopcion.html')
fig.show()

print("\nComparación de adopción (primeros 3 años):")
print(techs_comp)


Comparación de adopción (primeros 3 años):
        Tecnología  Año_1  Año_2  Año_3
0               3G   5190  21236  18504
1               4G  19367  31266  24481
2  5G (Proyección)  25177  40645  31825


## Resumen y Conclusiones

In [11]:
print("=" * 70)
print("PROYECCIÓN DE EXPANSIÓN 5G EN COLOMBIA (2025-2028)")
print("=" * 70)

print("\nMetodología:")
print("  - Modelo basado en curva de adopción 4G")
print("  - Factor de aceleración: 1.3x (tecnologías recientes se adoptan más rápido)")
print("  - Algoritmo: Gradient Boosting Regressor")

print("\nPredicciones Nacionales:")
for _, row in df_pred_5g.iterrows():
    año = int(row['AÑO'])
    centros = int(row['Centros_5G_predicho'])
    print(f"  {año}: {centros:,} centros con 5G")

total_2028 = df_pred_5g.iloc[-1]['Centros_5G_predicho']
total_centros = df_agg['Total_Centros'].sum() / len(df_agg['AÑO'].unique())
penetracion = (total_2028 / total_centros) * 100

print(f"\nPenetración proyectada 2028: {penetracion:.1f}%")

print("\nTop 3 departamentos proyectados para 2028:")
top_2028 = df_pred_5g_dept[df_pred_5g_dept['AÑO'] == 2028].nlargest(3, 'Centros_5G_predicho')
for _, row in top_2028.iterrows():
    print(f"  {row['DEPARTAMENTO']}: {int(row['Centros_5G_predicho']):,} centros")

crecimiento = df_pred_5g['Centros_5G_predicho'].pct_change() * 100
crecimiento_promedio = crecimiento[1:].mean()

print(f"\nCrecimiento anual promedio proyectado: {crecimiento_promedio:.1f}%")
print("=" * 70)

PROYECCIÓN DE EXPANSIÓN 5G EN COLOMBIA (2025-2028)

Metodología:
  - Modelo basado en curva de adopción 4G
  - Factor de aceleración: 1.3x (tecnologías recientes se adoptan más rápido)
  - Algoritmo: Gradient Boosting Regressor

Predicciones Nacionales:
  2025: 25,177 centros con 5G
  2026: 40,645 centros con 5G
  2027: 31,825 centros con 5G
  2028: 31,825 centros con 5G

Penetración proyectada 2028: 70.3%

Top 3 departamentos proyectados para 2028:
  CUNDINAMARCA: 4,090 centros
  ANTIOQUIA: 3,623 centros
  BOYACÁ: 2,238 centros

Crecimiento anual promedio proyectado: 13.2%
